In [ ]:
%load_ext autoreload
%autoreload 2


import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
from os.path import join as pjoin
import pandas as pd

In [ ]:
#os.environ['DATA_MEMORY_ERRORS_STAB_AND_STOCH']

In [ ]:
path_behav = '/home/demitau/memerr_code/exper_protocol/data'

fnbs = ['_debug__hypnotest_visuomotor_20240504_145324']
#'/home/demitau/ownCloud/Current/NeuroSci/Romain/code_memory_errors/exper_protocol/data/_debug__hypnotest_visuomotor_20240506_225706.log'
from behav_proc import readParamFiles, addBasicInfo, getGeomInfo

multi_param_read_mode = 'read_all'
#multi_param_read_mode = 'read_last'
print('Number of behav files = ',len(fnbs))

if multi_param_read_mode == 'read_last':    
    if len(fnbs) > 1:
        print('Selecting last param file')
    paramfiles = [fn for fn in list( os.listdir(path_behav) ) if fn.endswith('.param')]
    last = list( sorted( [ int(fn.split('_')[-1].split('.')[0]) for fn in paramfiles] ) )[-1]
    lastpf = [fn for fn in paramfiles if fn.find(str(last)) >= 0][0]
    fnbs = [lastpf.split('.')[0]]

dfs = []
for fnb in fnbs:
    fnp = pjoin(path_behav,fnb) + '.param'
    assert not fnb.endswith('.log')
    assert os.path.exists(fnp)
    print(fnp)
                
    params, phase2trigger, trigger2phase, CONTEXT_TRIGGER_DICT =\
        readParamFiles(fnp, path_behav)
    
    fn = fnb + '.log'
    fnp = fnb + '.param'

    fnf = pjoin(path_behav,fn)
    with open(fnf, 'r') as f:
        #lls = f.readlines()
        l = f.readline()
        truelen = len( l.split(',') )

    r = ('trial_index, current_phase_trigger, tgti_to_show,'
       ' vis_feedback_type, trial_type, special_block_type, block_ind, '
        ' feedbackX, feedbackY, unpert_feedbackX, unpert_feedbackY,'
         ' error_distance, target_coordX, target_coordY, '
         'feedbackX_when_crossing, feedbackY_when_crossing, '
         'jax1, jax2, reward, time, time_abs')
    r = r.replace(' ','')
    colnames = r.split(',')
    print('len(colnames) = ',len(colnames),'truelen = ',truelen,colnames)
    assert truelen == len(colnames), (truelen ,len(colnames))

    nbad = 3  # recompense strip
    #colnames = ['trial_index', 'current_phase_']
    df = pd.read_csv(pjoin(path_behav,fn), skipfooter = nbad, on_bad_lines='warn', header=0,
                    names=colnames, encoding='latin1')
    #encoding='latin-1'
    subj_ = fnb.split('_')[0]    
    df['subject'] = subj_
    dfs += [df]
    
    df['phase'] = df.apply(lambda row: trigger2phase[row['current_phase_trigger']], 1)
    
    fnf = pjoin(path_behav, fnb + '_trigger.log')
    from behav_proc import loadTriggerLog
    dftriglog = loadTriggerLog(fnf,CONTEXT_TRIGGER_DICT)

print( 'len = {}, maxtime in min = {}'.format( len(df), df['time'].max() / 60 ) )

In [ ]:
d = dict( zip(colnames, len(colnames)*['ignore it']) )
d['trial_index'] = 'index of the trial (starting from 0)'
d['block_ind'] = 'index of the block (starting from 0)'
d['vis_feedback_type'] = "pertrubation type (can be 'veridical', 'rot90', 'rot45', 'rot135')"
d['feedbackX'] = 'displayed feedback X coord (after applying perturbation), screen coords'
d['unpert_feedbackX'] = 'would-be true feedback X coord (i.e. before applying perturbation), screen coords'
d['target_coordX'] = 'target coordinates, screen coords'
d['jax1'] = 'joystick angle 1 (x direction)'
d['jax2'] = 'joystick angle 2 (y direction)'
d['reward'] = 'reward on this trial (zero in the beginning of trial, nonzero after reach is finished)'
d['time'] = 'time in seconds since start of the task'

print('Each row is a time frame. Beware that screen coordinates are not zero centered')
print('    Columns:')
for cni,cn in enumerate(colnames):
    #if len( d[cn] ):
    print('column ',cni,':', cn,' = ',d[cn])
#display(d)
d0 = d

I am concerned how to give analysis code to Eleonora:
teaching her to use all the thing is hard
I could either give her everything and a .py script to apply to data that would give her more columns
or I would have to simplify my code a lot to make it convertable

Another issue: with big angles there a jump over np.pi is happening and feedback becomes negative

In [ ]:
dfc['vis_feedback_type'].unique()

In [ ]:
from behav_proc import addBasicInfo
home_position, target_coords = getGeomInfo(params)

dfallph, dfc, dfcc, dfcpc, dfctc = addBasicInfo(df, phase2trigger, params,
                                             home_position, target_coords, def_subject_ind = 0,
                                               check_num_context_appearances=0)


In [ ]:
#                  subset of dfcc.columns
# 'ctxid', 'Nctx_app',
# 'error_endpoint_ang', 'error_lh_ang',
#        'error_unpert_lh_ang', 'ang_fb_lh', 'ang_ofb_lh',
#        'error_area2_signed_nn', 'error_unpert_area2_signed_nn', 'traj_length',
#        'ang_tgt', 'ang_fb_lh_to_postlh', 'error_lh2_ang',
#        'error_unpert_lh2_ang', 'error_lh_ang_deg', 'error_lh_ang_deg_abs',
#        'error_lh2_ang_deg', 'error_lh2_ang_deg_abs'
#     'error_area2_signed', 
#     'error_endpoint_ang_pert_adj', 'error_area2_signed_tln',
#        'error_area2_signed_nn_scaled_ed', 
           
#        'error_eucl', 
#     'feedback_tgt_unrel', 'org_feedback_tgt_unrel',
       
#         , 'time_mvt',
#        'traj_length_adj', 'error_pert_adj',
       
#     'prev_trial_type', 
    
#     'prev_time_lh',
#        'prev_perturbation', 'perturbation_diff', 'perturbation_diff_abs',
#        'prev_perturbation_valid', 
#     'perturbation_valid_diff',
#        'perturbation_valid_diff_abs', 
        
#        'prev_trial_index_valid', 'prev_block_pert'],
#       dtype='object')

In [ ]:
#params['target_location_spread']

In [ ]:
colnames2 = dfcc.columns

d = dict( zip(colnames2, len(colnames2)*['ignore it']) )
# angles by default are in radians, except for 'perturbation' columns (they are in deg)

print('Define SLTTTT to be straight line trajectory to the target starting at the starting point of the movement')
print('note that start position on each trial is slightly different due to imprecision of the joystick')
print('so I compute angle based on start position instead of based on center of center of the home circle')


d['subject'] = 'name of the subject'
d['Nctx_app'] = 'number of times participant experiencis the current perturbation'
d['time_lh'] = 'time since start of the trial when home circle is left (reaction time)'
d['time_l3h'] = 'same, but what is counted is time of going more than 3 home circle radii from the home circle'
d['nonhit'] = 'whether target was NOT hit'
d['ang_tgt'] = 'angle between the Nord direction and SLTTTT'
d['target_locs'] = 'angle of the target, counting from South direction CCW [deg]'
d['time_since_trial_start'] = 'self explanatory [sec]'
d['feedback'] = 'angle of the endpoint cursor position'
d['org_feedback'] = 'angle of the unperturbed endpoint cursor position'
d['prev_ctx_pert_same'] = 'whether previous perturbation was the same'
d['trialwb'] = 'index of trial within block '
d['traj_length'] = 'length of the trajectory (in pixels)'
d['traj_length_adj'] = 'length of the trajectory (in pixels) minus ideal trajectory length'
d['prev_block_pert'] = 'perturbation in the previous block'

d['ang_fb_lh'] = 'angle between the line segment connecting start position and the point on the home circle where the trajectory exited it, and SLTTTT'
d['error_lh2_ang'] = 'angle between the point on the home circle where the trajectory exited it and a point 10 frames after that (can be slightly more accurate than the previous angle), and SLTTTT'

d['error_area2_signed_nn'] = 'signed area (multiplied by a constant = 10. / area of 20 deg sector ) between the true trajectory and SLTTTT '
d['error_endpoint_ang'] = 'endpoint error in radians'
#ctxid?

#d['time_mvt'] = 'duration of movement (in milliseconds)'
d['error_lh2_ang_valid'] = 'setting errors on trials where the trial was not done in time to be equal to prev trial error'
#'error_unpert_lh2_ang_valid', 'error_area2_signed_nn_valid',
    
print('\nIn dfcc each row is a trial. Columns:')
for cni,cn in enumerate(colnames2):
    #if len( d[cn] ):
    if d[cn] == 'ignore it':
        del d[cn]
        continue
    print('column ',cni,':', cn,' = ',d[cn])
#display(d)

In [ ]:
#dfcc['error_lh2_ang']

In [ ]:
params['time_feedback']

In [ ]:
dfcc['time_mvt']

In [ ]:
cols_to_remove = set(dfcc.columns ) - set(d.keys()) - set(d0.keys())
cols_to_remove = list(sorted(cols_to_remove))
cols_to_remove

In [ ]:
dfcc.plot(x='trial_index',y=['perturbation','error_area2_signed_nn']) #error_lh_ang_deg

In [ ]:
dfcc.plot(x='trial_index',y=['perturbation','feedback']) #error_lh_ang_deg
#plt.axhline(0)

In [ ]:
dfcc.plot(x='trial_index',y=['perturbation','unpert_feedbackX','unpert_feedbackY']) #error_lh_ang_deg

In [ ]:
dfcc.plot(x='trial_index',y=['perturbation','org_feedback_tgt_unrel']) #error_lh_ang_deg

In [ ]:
dfcc.plot(x='trial_index',y=['perturbation','feedback','error_lh2_ang_deg']) #error_lh_ang_deg